In [1]:
#adding arrow keys

import numpy as np
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
from time import sleep
from pynput.keyboard import Controller, Key

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Initialize hand detector
detector = HandDetector(detectionCon=1)
keys = [["0","1","2","3","4","5","6","7","8","9"],
        ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

finalText = ""
keyboard = Controller()
shift_active = False
caps_lock_active = False

def drawALL(img, buttonList):
    imgNew = np.zeros_like(img, np.uint8)
    
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (x, y, w, h), 20, rt=0)
        if button.text in ["Shift", "Caps Lock"] and ((button.text == "Shift" and shift_active) or (button.text == "Caps Lock" and caps_lock_active)):
            cv2.rectangle(img, button.pos, (x+w, y+h), (0, 255, 0), cv2.FILLED)
        else:
            cv2.rectangle(img, button.pos, (x+w, y+h), (0, 130, 255), cv2.FILLED)
        if button.text in ["Enter", "Shift", "Backspace", "Caps Lock", "⇦", "⇨", "⇧", "⇩"]:
            cv2.putText(img, button.text, (x+10, y+60), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        else:
            cv2.putText(img, button.text, (x+20, y+75), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 4)
    
    out = img.copy()
    alpha = 0.5
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(img, alpha, imgNew, 1 - alpha, 0)[mask]    
    return out

class Button():
    def __init__(self, pos, text, size=[85, 85]):
        self.pos = pos
        self.text = text
        self.size = size

buttonList = []
for i in range(len(keys)):
    for x, key in enumerate(keys[i]):
        buttonList.append(Button([100*x+50, 100*i+50], key))

# Add space bar
spaceBarWidth = 800
spaceBarHeight = 85
spaceBarX = (1280 - spaceBarWidth) // 2
spaceBarY = 100 * len(keys) + 50
buttonList.append(Button([spaceBarX, spaceBarY], "Space", [spaceBarWidth, spaceBarHeight]))

# Add enter key
enterKeyWidth = 150
enterKeyHeight = 185
enterKeyX = 1100
enterKeyY = 150
buttonList.append(Button([enterKeyX, enterKeyY], "Enter", [enterKeyWidth, enterKeyHeight]))

# Add backspace key
backspaceKeyWidth = 150
backspaceKeyHeight = 85
backspaceKeyX = 1100
backspaceKeyY = 50
buttonList.append(Button([backspaceKeyX, backspaceKeyY], "Backspace", [backspaceKeyWidth, backspaceKeyHeight]))

# Add shift key
shiftKeyWidth = 150
shiftKeyHeight = 85
shiftKeyX = 1100
shiftKeyY = 350
buttonList.append(Button([shiftKeyX, shiftKeyY], "Shift", [shiftKeyWidth, shiftKeyHeight]))

# Add caps lock key
capsLockKeyWidth = 150
capsLockKeyHeight = 85
capsLockKeyX = 1100
capsLockKeyY = 450
buttonList.append(Button([capsLockKeyX, capsLockKeyY], "Caps Lock", [capsLockKeyWidth, capsLockKeyHeight]))

# Add arrow keys
arrowKeySize = 85
leftArrowX, leftArrowY = 950, 600
downArrowX, downArrowY = 1100 + arrowKeySize, 550
upArrowX, upArrowY = 1100 + arrowKeySize, 550 - arrowKeySize
rightArrowX, rightArrowY = 1100 + 2*arrowKeySize, 550

buttonList.append(Button([leftArrowX, leftArrowY], "L", [arrowKeySize, arrowKeySize]))
buttonList.append(Button([downArrowX, downArrowY], "D", [arrowKeySize, arrowKeySize]))
buttonList.append(Button([upArrowX, upArrowY], "U", [arrowKeySize, arrowKeySize]))
buttonList.append(Button([rightArrowX, rightArrowY], "R", [arrowKeySize, arrowKeySize]))

# Initialize variables for click detection
clickedButton = None
clickCooldown = 0

while True:
    success, img = cap.read()
    img = detector.findHands(img)  # This will find and draw the hands
    lmList, bboxInfo = detector.findPosition(img)  # This retrieves the landmarks and bounding box
    img = drawALL(img, buttonList)
    
    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size
            if x < lmList[8][0] < x+w and y < lmList[8][1] < y+h:
                cv2.rectangle(img, button.pos, (x+w, y+h), (150, 130, 255), cv2.FILLED)
                if button.text in ["Enter", "Shift", "Backspace", "Caps Lock", "⇦", "⇨", "⇧", "⇩"]:
                    cv2.putText(img, button.text, (x+10, y+60), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
                else:
                    cv2.putText(img, button.text, (x+20, y+75), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 4)
                l, _, _ = detector.findDistance(8, 12, img, draw=False)
                
                if l < 30:
                    cv2.rectangle(img, button.pos, (x+w, y+h), (0, 255, 0), cv2.FILLED)
                    if button.text in ["Enter", "Shift", "Backspace", "Caps Lock", "⇦", "⇨", "⇧", "⇩"]:
                        cv2.putText(img, button.text, (x+10, y+60), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
                    else:
                        cv2.putText(img, button.text, (x+20, y+75), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 4)
                    if clickCooldown == 0:
                        if button.text == "Space":
                            keyboard.press(" ")
                            finalText += " "
                        elif button.text == "Enter":
                            keyboard.press(Key.enter)
                            finalText += "\n"
                        elif button.text == "Backspace":
                            keyboard.press(Key.backspace)
                            finalText = finalText[:-1]
                        elif button.text == "Shift":
                            shift_active = not shift_active
                        elif button.text == "Caps Lock":
                            caps_lock_active = not caps_lock_active
                        elif button.text in ["L", "R", "U", "D"]:
                            if button.text == "L":
                                keyboard.press(Key.left)
                            elif button.text == "R":
                                keyboard.press(Key.right)
                            elif button.text == "U":
                                keyboard.press(Key.up)
                            elif button.text == "D":
                                keyboard.press(Key.down)
                        else:
                            if shift_active or caps_lock_active:
                                finalText += button.text.upper()
                                keyboard.press(button.text.upper())
                                shift_active = False
                            else:
                                finalText += button.text.lower()
                                keyboard.press(button.text.lower())
                        clickCooldown = 20  # Set cooldown to prevent multiple clicks

    if clickCooldown > 0:
        clickCooldown -= 1
    
    # Split finalText into lines and display them
    lines = finalText.split('\n')
    y_offset = 600
    for line in lines:
        cv2.putText(img, line, (500, y_offset), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)
        y_offset += 40  # Adjust this value to change line spacing
    

    cv2.imshow("Image", img)
    cv2.waitKey(1)

c:\Users\sp_kr\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [1]:
#adding a parallel symbols keyboard
#not showing capslock and shift keys on symbol board

import numpy as np
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
from time import sleep
from pynput.keyboard import Controller, Key

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Initialize hand detector
detector = HandDetector(detectionCon=1)
keys_alphabet = [["0","1","2","3","4","5","6","7","8","9"],
        ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

keys_symbols = [["0","1","2","3","4","5","6","7","8","9"],
        ["+", "-", "*", "/", "=", ".", "{", "}", "[", "]"],
        ["!", "@", "#", "$", "%", "^", "&", "*", "(", ")"],
        ["`", "~", "'", '"', ":", ";", ",", "?", "<", ">"]]

finalText = ""
keyboard = Controller()
shift_active = False
caps_lock_active = False
symbols_active = False

def drawALL(img, buttonList):
    imgNew = np.zeros_like(img, np.uint8)
    
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (x, y, w, h), 20, rt=0)
        if button.text in ["Shift", "Caps Lock", "TAB", "Symbols", "Letters"] and ((button.text == "Shift" and shift_active) or (button.text == "Caps Lock" and caps_lock_active) or (button.text in ["Symbols", "Letters"] and symbols_active)):
            cv2.rectangle(img, button.pos, (x+w, y+h), (0, 255, 0), cv2.FILLED)
        else:
            cv2.rectangle(img, button.pos, (x+w, y+h), (255, 120, 120), cv2.FILLED)
        if button.text in ["Enter", "Shift", "Backspace", "Caps Lock", "TAB", "Left", "Right", "Up", "Down", "Symbols", "Letters"]:
            cv2.putText(img, button.text, (x+10, y+37), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
        else:
            cv2.putText(img, button.text, (x+15, y+50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
    
    out = img.copy()
    alpha = 0.5
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(img, alpha, imgNew, 1 - alpha, 0)[mask]    
    return out

class Button():
    def __init__(self, pos, text, size=[60, 60]):
        self.pos = pos
        self.text = text
        self.size = size

def createButtonList(keys, include_caps_lock=True,include_shift = True):
    buttonList = []
    for i in range(len(keys)):
        for x, key in enumerate(keys[i]):
            buttonList.append(Button([80*x+140, 80*i+120], key))
    
    # Add space bar
    spaceBarWidth = 500
    spaceBarHeight = 60
    spaceBarX = 280
    spaceBarY = 100 * len(keys) + 50
    buttonList.append(Button([spaceBarX, spaceBarY], "Space", [spaceBarWidth, spaceBarHeight]))

    # Add enter key
    enterKeyWidth = 105
    enterKeyHeight = 60
    enterKeyX = 940
    enterKeyY = 200
    buttonList.append(Button([enterKeyX, enterKeyY], "Enter", [enterKeyWidth, enterKeyHeight]))

    # Add backspace key
    backspaceKeyWidth = 105
    backspaceKeyHeight = 60
    backspaceKeyX = 940
    backspaceKeyY = 120
    buttonList.append(Button([backspaceKeyX, backspaceKeyY], "Backspace", [backspaceKeyWidth, backspaceKeyHeight]))

    # Add shift key

    # Add caps lock key only for alphabet layout
    if include_caps_lock:
        capsLockKeyWidth = 105
        capsLockKeyHeight = 60
        capsLockKeyX = 15
        capsLockKeyY = 200
        buttonList.append(Button([capsLockKeyX, capsLockKeyY], "Caps Lock", [capsLockKeyWidth, capsLockKeyHeight]))
    if include_shift:
            shiftKeyWidth = 105
            shiftKeyHeight = 60
            shiftKeyX = 15
            shiftKeyY = 120
            buttonList.append(Button([shiftKeyX, shiftKeyY], "Shift", [shiftKeyWidth, shiftKeyHeight]))

    # Add TAB key
    tabKeyWidth = 105
    tabKeyHeight = 60
    tabKeyX = 15
    tabKeyY = 280
    buttonList.append(Button([tabKeyX, tabKeyY], "TAB", [tabKeyWidth, tabKeyHeight]))

    # Add Symbols/Letters key
    symbolsKeyWidth = 105
    symbolsKeyHeight = 60
    symbolsKeyX = 15
    symbolsKeyY = 360
    buttonList.append(Button([symbolsKeyX, symbolsKeyY], "Symbols" if not symbols_active else "Letters", [symbolsKeyWidth, symbolsKeyHeight]))

    # Add arrow keys
    arrowKeySize = 60
    leftArrowX, leftArrowY = 900, 100 * len(keys) + 50
    downArrowX, downArrowY = 900 + arrowKeySize, 100 * len(keys) + 50
    upArrowX, upArrowY = 900 + arrowKeySize, (100 * len(keys) + 50) - (arrowKeySize)
    rightArrowX, rightArrowY = 900 + 2*arrowKeySize, 100 * len(keys) + 50

    buttonList.append(Button([leftArrowX, leftArrowY], "Left", [arrowKeySize, arrowKeySize]))
    buttonList.append(Button([downArrowX, downArrowY], "Down", [arrowKeySize, arrowKeySize]))
    buttonList.append(Button([upArrowX, upArrowY], "Up", [arrowKeySize, arrowKeySize]))
    buttonList.append(Button([rightArrowX, rightArrowY], "Right", [arrowKeySize, arrowKeySize]))

    return buttonList

buttonList = createButtonList(keys_alphabet)

# Initialize variables for click detection
clickedButton = None
clickCooldown = 0

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img = detector.findHands(img)  # This will find and draw the hands
    lmList, bboxInfo = detector.findPosition(img)  # This retrieves the landmarks and bounding box
    img = drawALL(img, buttonList)
    
    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size
            if x < lmList[8][0] < x+w and y < lmList[8][1] < y+h:
                cv2.rectangle(img, button.pos, (x+w, y+h), (150, 130, 255), cv2.FILLED)
                if button.text in ["Enter", "Shift", "Backspace", "TAB", "Caps Lock", "Left", "Right", "Up", "Down", "Symbols", "Letters"]:
                    cv2.putText(img, button.text, (x+10, y+37), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
                else:
                    cv2.putText(img, button.text, (x+15, y+50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
                l, _, _ = detector.findDistance(8, 12, img, draw=False)
                
                if l < 30:
                    cv2.rectangle(img, button.pos, (x+w, y+h), (0, 255, 0), cv2.FILLED)
                    if button.text in ["Enter", "Shift", "Backspace", "TAB", "Caps Lock", "Left", "Right", "Up", "Down", "Symbols", "Letters"]:
                        cv2.putText(img, button.text, (x+10, y+37), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
                    else:
                        cv2.putText(img, button.text, (x+15, y+50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
                    if clickCooldown == 0:
                        if button.text == "Space":
                            keyboard.press(" ")
                            finalText += " "
                        elif button.text == "TAB":
                            keyboard.press(Key.tab)
                            finalText += "    "
                        elif button.text == "Enter":
                            keyboard.press(Key.enter)
                            finalText += "\n"
                        elif button.text == "Backspace":
                            keyboard.press(Key.backspace)
                            finalText = finalText[:-1]
                        elif button.text == "Shift":
                            shift_active = not shift_active
                        elif button.text == "Caps Lock":
                            caps_lock_active = not caps_lock_active
                        elif button.text in ["Left", "Right", "Up", "Down"]:
                            if button.text == "Left":
                                keyboard.press(Key.left)
                            elif button.text == "Right":
                                keyboard.press(Key.right)
                            elif button.text == "Up":
                                keyboard.press(Key.up)
                            elif button.text == "Down":
                                keyboard.press(Key.down)
                        elif button.text in ["Symbols", "Letters"]:
                            symbols_active = not symbols_active
                            if symbols_active:
                                buttonList = createButtonList(keys_symbols, include_caps_lock=False,include_shift = False)
                            else:
                                buttonList = createButtonList(keys_alphabet, include_caps_lock=True, include_shift=True)
                        else:
                            if shift_active or caps_lock_active:
                                finalText += button.text.upper()
                                keyboard.press(button.text.upper())
                                shift_active = False
                            else:
                                finalText += button.text.lower()
                                keyboard.press(button.text.lower())
                        clickCooldown = 10  # Set cooldown to prevent multiple clicks

    if clickCooldown > 0:
        clickCooldown -= 1
    
    # Split finalText into lines and display them
    lines = finalText.split('\n')
    y_offset = 600
    for line in lines:
        cv2.putText(img, line, (500, y_offset), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)
        y_offset += 40  # Adjust this value to change line spacing
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)

c:\Users\sp_kr\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
